# Семинар 4: ` Keras` и сверточные нейронные сети

На семинаре мы при помощи библиотеки `keras` обучим глубокую сверточную нейронную сеть.

Библиотека `keras` — надстройка над фреймворками [Tensorflow](https://www.tensorflow.org/), [Theano](http://deeplearning.net/software/theano/), [CNTK](https://docs.microsoft.com/en-us/cognitive-toolkit/). Можно использовать в качестве бэкенда любой из этих фреймворков, и код будет исполняться независмо от выбранного бэкенда. По умолчанию `keras` работает с `tensorflow`.

In [20]:
import keras
keras.backend.backend()


'tensorflow'

In [21]:
import tensorflow as tf
from keras import backend as K

# соответствует tf.placeholder() или th.tensor.matrix(), th.tensor.tensor3(), и т. д. в зависимости от бэкенда
inputs = K.placeholder(shape=(2, 4, 5))
inputs = K.placeholder(shape=(None, 4, 5))
inputs = K.placeholder(ndim=3)

# делать операции над тензорами можно как мы привыкли с tensorflow
b = K.random_uniform_variable(shape=(4, 5), low=0, high=1)
c = K.random_normal_variable(shape=(4, 5), mean=0, scale=1)

a = b + c * K.abs(b)
c = K.dot(a, K.transpose(b))
d = K.sum(b, axis=1)
b = K.concatenate([b, c], axis=-1)

# в результате операций над тензорами получили неинициализированный тензор, как и для tensorflow
print(c, "\n")

# а теперь посчитаем, чему будут равны тензоры для конкретных входных данных
print("c: \n", K.eval(c))
print("d: \n", K.eval(d))
print("b: \n", K.eval(b))

K.clear_session()

Tensor("MatMul:0", shape=(4, 4), dtype=float32) 

c: 
 [[3.338108   2.4369338  3.3162317  3.0676663 ]
 [1.2508358  1.4171395  1.4698844  1.8300776 ]
 [0.37694338 0.37492004 0.67462504 0.37973586]
 [1.8413678  2.4478116  2.1607587  2.2805357 ]]
d: 
 [2.5633025 2.6991744 3.0342622 3.4339776]
b: 
 [[0.6868795  0.10215402 0.9560753  0.2799462  0.53824747 3.338108
  2.4369338  3.3162317  3.0676663 ]
 [0.7791878  0.3839116  0.37285638 0.68732035 0.47589815 1.2508358
  1.4171395  1.4698844  1.8300776 ]
 [0.53053725 0.20323586 0.86687803 0.7380997  0.6955112  0.37694338
  0.37492004 0.67462504 0.37973586]
 [0.72693145 0.9145076  0.7043344  0.6261914  0.4620129  1.8413678
  2.4478116  2.1607587  2.2805357 ]]


# Keras Sequential API
Основная структура данных в `Keras` —  это  `model`, способ организовать слои в нейронной сети. Простейший способ создать модель — использовать `Sequential()`. Такая модель представляет из себя стек слоев, идущих последовательно друг за другом, именно такой моделью мы и будем сегодня пользоваться для обучения сверточной нейронной сети.

In [25]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

model = Sequential()

# для первого слоя  в последовательности нужно указать размерности входных данных
# для других слоев делать это уже не нужно
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(20, activation='relu'))
model.add(Dense(10, activation='softmax'))

# компилируем модель, определяем функцию потерь, метрику качества, метод оптимизации
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(10, size=(1000, 1))

one_hot_labels = to_categorical(labels, num_classes=10)

# обучаем модель, указываем количество шагов градиентного спуска, размер батча
model.fit(data, one_hot_labels, epochs=10, batch_size=32, verbose=0)

TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("Placeholder_3:0", shape=(), dtype=float32) is not an element of this graph.

# Keras Callbacks
Callbacks — это набор функций, которые можно передать в процедуру обучения модели. Можно использовать колбэки, чтобы следить за метриками во время обучения, что может быть полезно для дебага, сохранять модель и так далее. Список колбэков передается в `.fit()`, релевантные колбэки вызываются на каждой итерации обучения.

В `keras` реализовано [много полезных колбэков](https://keras.io/callbacks/), так же есть возможность писать кастомные.

In [8]:
# определим кастомный колбэк, который выводит функции потерь для каждого батча на каждой итерации
class LossHistory(keras.callbacks.Callback):        
    def on_epoch_begin(self, epoch, logs={}):
        # в self.losses будем хранить искомый список функций потерь для батчей, надо его инициализировать
        # еще можно вывести номер итерации
        ### YOUR CODE HERE
        self.losses = []
        print("Epoch:", epoch)
        
    def on_epoch_end(self, epoch, logs={}):
        # выполнили один шаг градиентного спуска, самое время вывести вычисленные функции потерь
        ### YOUR CODE HERE 
        print("Batch losses:", self.losses)
        print("Loss:", logs.get('loss'))

    def on_batch_end(self, batch, logs={}):
        # закончили обрабатывать батч, надо сохранить лосс, он лежит в logs.get('loss')
        ### YOUR CODE HERE 
        self.losses.append(logs.get('loss'))


history = LossHistory()
model.fit(data,
          one_hot_labels,
          epochs=10,
          batch_size=200,
          verbose=0,
          callbacks=[keras.callbacks.EarlyStopping(monitor='loss'), # добавим еще EarlyStopping, реализованный в keras
                     history]
         )

Epoch: 0


AttributeError: 'EarlyStopping' object has no attribute 'on_train_batch_begin'

# Сверточная нейронная сеть для CIFAR-10

Датасет CIFAR-10 содержит цветные картинки 32x32, каждая из которых принадлежит одному из 10 классов: __airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck__

В задании нужно будет:
* Определить архитектуру CNN
* Обучить модель
* Визуализировать обученные фильтры


**ОБРАТИТЕ ВНИМАНИЕ:** обучение CNN на CPU для этого задания может занимать достаточно много времени, 0.5-1.5 часа, поэтому будем обучать модель на GPU

**Runtime -> Change runtime type** и выберите **GPU** в Google Colab


In [0]:
import tensorflow as tf
import keras
from keras import backend as K
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
print(tf.__version__)
print(keras.__version__)

In [0]:
# чистим сессию, чтобы не получить out-of-memory
def reset_tf_session():
    curr_session = tf.get_default_session()
    if curr_session is not None:
        curr_session.close()
    K.clear_session()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    s = tf.InteractiveSession(config=config)
    K.set_session(s)
    return s

# Загрузка датасета

In [0]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [0]:
print("Train samples:", x_train.shape, y_train.shape)
print("Test samples:", x_test.shape, y_test.shape)

In [0]:
NUM_CLASSES = 10
cifar10_classes = ["airplane", "automobile", "bird", "cat", "deer", 
                   "dog", "frog", "horse", "ship", "truck"]

In [0]:
# посмотрим на случайные картинки из датасета
cols = 8
rows = 2
fig = plt.figure(figsize=(2 * cols - 1, 2.5 * rows - 1))
for i in range(cols):
    for j in range(rows):
        random_index = np.random.randint(0, len(y_train))
        ax = fig.add_subplot(rows, cols, i * rows + j + 1)
        ax.grid(False)
        ax.axis('off')
        ax.imshow(x_train[random_index, :])
        ax.set_title(cifar10_classes[y_train[random_index, 0]])
plt.show()

# Нормализация данных
Как мы знаем, нейронные сети любят, когда входные данные нормализованы, нормализуем их: $$x_{norm} = \frac{x}{255} - 0.5$$

Так же нужно преобразовать классы в one-hot encoded векторы с помощью __keras.utils.to_categorical__.

In [0]:
# нормализуем входные данные
x_train2 = ### YOUR CODE HERE
x_test2 = ### YOUR CODE HERE

# преобразуем метки классов в one-hot encoded векторы, должно получиться(?, NUM_CLASSES)
y_train2 = ### YOUR CODE HERE
y_test2 = ### YOUR CODE HERE

# Архитектура CNN

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from keras.layers.advanced_activations import LeakyReLU

Сверточные нейронные сети строятся из нескольких типов слоев:
- [Conv2D](https://keras.io/layers/convolutional/#conv2d) - свертка:
    - **filters**: количество выходных каналов; 
    - **kernel_size**: ширина и высота окна для свертки, tuple/list из двух целых чисел;
    - **padding**: padding="same" добавляет столько нулей, чтобы выходной канал имел такие же размеры, как и вход, при padding='valid' свертка выполняется только там, где ядро свертки полностью пересекается с входным каналом;
    - **activation**: "relu", "tanh", и т.д.
    - **input_shape**: размерности входных данных.
- [MaxPooling2D](https://keras.io/layers/pooling/#maxpooling2d) - выполняет 2D max pooling.
- [Flatten](https://keras.io/layers/core/#flatten) - не затрагивает размерность,  соответствующую размеру батча.
- [Dense](https://keras.io/layers/core/#dense) - полносвязный слой.
- [Activation](https://keras.io/layers/core/#activation) - применяет функцию активации.
- [LeakyReLU](https://keras.io/layers/advanced-activations/#leakyrelu) - применяет функцию активации leaky relu.
- [Dropout](https://keras.io/layers/core/#dropout) - применяет dropout.

Размерность входных данных для модели __(None, 32, 32, 3)__ , размерность выходных __(None, 10)__ . __None__ соответствует размерности размера батча.

Шаблон модели:

```python
model = Sequential() 
model.add(Conv2D(..., input_shape=(32, 32, 3)))  # не забываем определить размерность входа для первого слоя

...  # сверточные слои, пулинг, дропаут

model.add(Dense(NUM_CLASSES))  
model.add(Activation("softmax"))
```

In [0]:
def make_model():
    """
    Define your model architecture here.
    Returns `Sequential` model.
    """
    
    ### YOUR CODE HERE
    
    model = Sequential()
      
    return model


In [0]:
s = reset_tf_session()
model = make_model()
model.summary()

# Обучение модели

На GPU 10-20 секунд на эпоху.
Добавим в обучение уменьшение learning rate с номером эпохи при помощи callback

In [0]:
INIT_LR = 5e-3  # стартовый learning rate
BATCH_SIZE = 32
EPOCHS = 10

s = reset_tf_session()

model = make_model()

# определите функцию потреть, метод оптимизации и метрику качества
### YOUR CODE HERE

# формула, по которой будет меняться learning rate в зависимости от эпохи, передадим ее в LearningRateScheduler callback
def lr_scheduler(epoch):
    return INIT_LR * 0.9 ** epoch

# будем выводить актуальный learning rate в начале каждой итерации (on_epoch_begin)
# для этого нужно написать кастомный callback
class LrHistory(keras.callbacks.Callback):
    ### YOUR CODE HERE


In [0]:
# обучаем модель
model.fit(
    x_train2, y_train2,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[keras.callbacks.LearningRateScheduler(lr_scheduler), 
               LrHistory(),],
    validation_data=(x_test2, y_test2),
    shuffle=True,
    verbose=1,
    initial_epoch=0    
)

In [0]:
# сохраняем веса модели
model.save_weights("weights.h5")

In [0]:
# загружаем веса (можно не вызывать model.fit, если модель была однажды обучена и веса сохранены)
model.load_weights("weights.h5")

# Применение модели

In [0]:
# вычисляем прогноз для тестовых данных
y_pred_test = model.predict_proba(x_test2)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)
y_pred_test_max_probas = np.max(y_pred_test, axis=1)

In [0]:
# матрица ошибок и точность
from sklearn.metrics import confusion_matrix, accuracy_score
plt.figure(figsize=(7, 6))
plt.title('Confusion matrix', fontsize=16)
plt.imshow(confusion_matrix(y_test, y_pred_test_classes))
plt.xticks(np.arange(10), cifar10_classes, rotation=45, fontsize=12)
plt.yticks(np.arange(10), cifar10_classes, fontsize=12)
plt.colorbar()
plt.show()
print("Test accuracy:", accuracy_score(y_test, y_pred_test_classes))

In [0]:
# посмотрим на предсказания
cols = 8
rows = 2
fig = plt.figure(figsize=(2 * cols - 1, 3 * rows - 1))
for i in range(cols):
    for j in range(rows):
        random_index = np.random.randint(0, len(y_test))
        ax = fig.add_subplot(rows, cols, i * rows + j + 1)
        ax.grid(False)
        ax.axis("off")
        ax.imshow(x_test[random_index, :])
        pred_label = cifar10_classes[y_pred_test_classes[random_index]]
        pred_proba = y_pred_test_max_probas[random_index]
        true_label = cifar10_classes[y_test[random_index, 0]]
        ax.set_title("pred: {}\nscore: {:.3}\ntrue: {}".format(
               pred_label, pred_proba, true_label
        ))
plt.show()

# Визуализация фильтров

Мы хотим найти входные изображения, максимизирующие активации для заданного слоя нейронной сети.

Задача очень похожа на задачу обучения весов для нейронной сети: при обучении весов нам нужно найти веса, минимизирующие функцию потерь, для данного входного изображения. Решаем эту задачу градиентным спуском, ищем градиент функции потерь по весам.

Теперь мы хотим найти входное изображение, максимизирующее среднее значение активации на заданном слое
$$Loss(layer) = Mean(layer) = \frac{1}{|layer|}\sum_{x \in layer}x \rightarrow \max    $$

Задачи максимизации решаются аналогично задачам минимизации с помощью градиентного подъема

$$image_{t+1} = Image_t + \gamma\frac{\partial  Loss}{\partial   Image_t}$$


In [0]:
s = reset_tf_session()
K.set_learning_phase(0)
model = make_model()
model.load_weights("weights.h5")  # веса сохранены после обучения model.fit

In [0]:
model.summary()

In [0]:
def find_maximum_stimuli(layer_name, is_conv, filter_index, model, iterations=20, step=1., verbose=True):
    
    def image_values_to_rgb(x):
        # нормализуем x: центрируем в 0 (np.mean(x_train2)), убедиться, что std=0.25 (np.std(x_train2))
        # теперь х такой же как нормализованный вход нейронной сети
        x = ### YOUR CODE HERE        

        # преобразуем нормализованный вход обратно к RGB: x = (x_norm + 0.5) * 255
        x = ### YOUR CODE HERE
    
        x = np.clip(x, 0, 255).astype('uint8')
        return x

    # placeholder для входного изображения
    input_img = model.input
    img_width, img_height = input_img.shape.as_list()[1:3]
    
    # находим выходы заданного слоя
    layer_output = list(filter(lambda x: x.name == layer_name, model.layers))[0].output
    
    # определите функцию потерь, обратите внимание, что мы можем считать ее 
    # как для сверточного слоя, так и для полносвязного

    loss = ### YOUR CODE HERE

    # посчитайте градиент функции потерь по входному изображению с помощью  K.gradients

    grads = ### YOUR CODE HERE

    # normalization trick
    grads = grads / (K.sqrt(K.sum(K.square(grads))) + 1e-10)

    # считаем градиенты
    iterate = K.function([input_img], [loss, grads])

    # инициализируем входное ихображение: серое изображение со случайным шумом
    input_img_data = np.random.random((1, img_width, img_height, 3))
    input_img_data = (input_img_data - 0.5) * (0.1 if is_conv else 0.001)

    # градиентный подъем
    for i in range(iterations):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step
        if verbose:
            print('Current loss value:', loss_value)

    # преобразуем изображение в RGB формат
    img = image_values_to_rgb(input_img_data[0])
    
    return img, loss_value

In [0]:
# sample maximum stimuli
def plot_filters_stimuli(layer_name, is_conv, model, iterations=20, step=1., verbose=False):
    cols = 8
    rows = 2
    filter_index = 0
    max_filter_index = list(filter(lambda x: x.name == layer_name, model.layers))[0].output.shape.as_list()[-1] - 1
    fig = plt.figure(figsize=(2 * cols - 1, 3 * rows - 1))
    for i in range(cols):
        for j in range(rows):
            if filter_index <= max_filter_index:
                ax = fig.add_subplot(rows, cols, i * rows + j + 1)
                ax.grid(False)
                ax.axis('off')
                loss = -1e20
                while loss < 0 and filter_index <= max_filter_index:
                    stimuli, loss = find_maximum_stimuli(layer_name, is_conv, filter_index, model,
                                                         iterations, step, verbose=verbose)
                    filter_index += 1
                if loss > 0:
                    ax.imshow(stimuli)
                    ax.set_title("Filter #{}".format(filter_index))
    plt.show()

In [0]:
# maximum stimuli for convolutional neurons
conv_activation_layers = []
for layer in model.layers:
    if isinstance(layer, LeakyReLU):
        prev_layer = layer._inbound_nodes[0].inbound_layers[0]
        if isinstance(prev_layer, Conv2D):
            conv_activation_layers.append(layer)

for layer in conv_activation_layers:
    print(layer.name)
    plot_filters_stimuli(layer_name=layer.name, is_conv=True, model=model)

In [0]:
# maximum stimuli for last dense layer
last_dense_layer = list(filter(lambda x: isinstance(x, Dense), model.layers))[-1]
plot_filters_stimuli(layer_name=last_dense_layer.name, is_conv=False, 
                     iterations=200, step=0.1, model=model)